In [ ]:
# Instalando o scikit-surprise
!pip3 install scikit-surprise

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from functools import reduce 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate, KFold

import warnings; warnings.simplefilter('ignore')

# **RECOMENDADOR BASEADO EM CONTEÚDO**

In [ ]:
# Carregando o dataset movies_metadata
md_filmes = pd.read_csv('movies_metadata.csv')
# Exibindo metadados dos filmes
md_filmes

In [ ]:
md_filmes['genres'] = md_filmes['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance (x, list) else [])

In [ ]:
md_filmes['year'] = pd.to_datetime(md_filmes['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [ ]:
md_filmes.head()

In [ ]:
# Carregando dataset links_small
links_small = pd.read_csv('links_small.csv', dtype={'movieId':"Int64",'imdbId':"Int64",'tmdbId':"Int64"})
# Vai ser usado no KNN
ds_links = links_small
# Converte o tipo da coluna, filtra os não nulos e cria uma serie com os valores de tmdbId
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
# Exibindo a serie 
links_small

In [ ]:
# Essas linhas possuem dados maus formatados, por isso foram removidas,
# o rest_index é para remover a coluna de index que foi criada, pois a 
# mesma estava fora de ordem 
md_filmes = md_filmes.drop([19730, 29503, 35587])
# Exibindo metadados dos filmes
md_filmes

In [ ]:
md_filmes[md_filmes['title'] == 'Black Gold'][['popularity']]

In [ ]:
# Convete o tipo da coluna 
md_filmes['id'] = md_filmes['id'].astype('int')

In [ ]:
# Convete o tipo da coluna popularidade para float
md_filmes['popularity'] = md_filmes['popularity'].astype('float')

In [ ]:
# Para conseguir remover as duplicatas 
md_filmes['genres'] = md_filmes['genres'].astype('str')

In [ ]:
# Ordenando coluna de popularidade
md_filmes = md_filmes.sort_values('popularity', ascending=False).drop_duplicates().sort_index()

In [ ]:
md_filmes

In [ ]:
# Removendo os duplicados que contém mesmo valore de id, imdb_id, title
md_filmes = md_filmes.drop_duplicates(subset=['id', 'imdb_id', 'title'])

In [ ]:
# Convete o tipo da coluna genero para list
md_filmes['genres'] = md_filmes['genres'].apply(literal_eval)

In [ ]:
md_filmes[md_filmes['title'] == 'Black Gold'][['popularity']]

### ***RECOMENDADOR BASEADO EM METADADOS DOS FILMES***

In [ ]:
# Carregando dados dos credits  
credits = pd.read_csv('credits.csv')
# Carregando dados dos keywords  
keywords = pd.read_csv('keywords.csv')

In [ ]:
# Converte o tipo da coluna para int
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md_filmes['id'] = md_filmes['id'].astype('int')

In [ ]:
md_filmes.shape

In [ ]:
# Fazendo a junção dos Dataframes de filmes com creditos e palavras chave 
md_filmes = md_filmes.merge(credits, on='id')
md_filmes = md_filmes.merge(keywords, on='id')

In [ ]:
# Para conseguir remover as duplicatas 
md_filmes['genres'] = md_filmes['genres'].astype('str')

In [ ]:
# Removendo duplicados
md_filmes = md_filmes.drop_duplicates()

In [ ]:
md_filmes

In [ ]:
# Convete o tipo da coluna genero para list
md_filmes['genres'] = md_filmes['genres'].apply(literal_eval)

In [ ]:
# Colete todos os metadados para filmes no conjunto de dados links_small
# Filtre os filmes cujo id está presente no arquivo links.csv e armazene em smd
# .isin -> Se cada elemento no DataFrame está contido em valores.
# Similar movie data - smd
smd = md_filmes[md_filmes['id'].isin(links_small)]
# ds_filmes vai ser usado no KNN
ds_filmes = smd
smd.shape

In [ ]:
smd

In [ ]:
smd['cast'] = smd['cast'].apply(literal_eval)

In [ ]:
smd['crew'] = smd['crew'].apply(literal_eval)

In [ ]:
smd['keywords'] = smd['keywords'].apply(literal_eval)

In [ ]:
# Cria nova coluna com tamanho do cast
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))

In [ ]:
# Cria nova coluna com tamanho do crew
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [ ]:
smd.head()

In [ ]:
def obter_diretor(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [ ]:
smd['director'] = smd['crew'].apply(obter_diretor)

In [ ]:
# Organiza o nome do cast
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
# Seleciona os 3 principais atores 
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [ ]:
smd.head()

In [ ]:
smd['keywords'][1]

In [ ]:
# Organiza as palavras chaves numa lista
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) 

In [ ]:
# Remove os espaços e deixa tudo minusculo no cast
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
# Converte o tipo da coluna do director para string, pega o nome do diretor deixa minusculo e sem espaços
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [ ]:
# Repete 3 vezes o nome do diretor na lista para dar mais peso em relação a todo elenco
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

#### **PALAVRAS-CHAVE**  
Será feito um pré-processamento das palavras chaves

In [ ]:
smd

In [ ]:
# Criar uma serie palavras chave aplicando ao longo do axis=1 coluna
# stack ->retornar um dataframe remodelado, se as colunas tiverem um único nível, a saída é uma Série;
# Reseta index em 1 nível
palavras_chave = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)

In [ ]:
# Nomeia a serie
palavras_chave.name = 'keyword'

In [ ]:
# Retorna uma série contendo contagens de valores exclusivos.
palavras_chave = palavras_chave.value_counts()

#Exibindo as 5 palavras que ocorrem com mais frequencia  
palavras_chave[:5]

In [ ]:
# Selecionando somente as palavras que ocorrem mais de uma vez
palavras_chave = palavras_chave[palavras_chave > 1]

In [ ]:
# Converterndo palavras em seu radical
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

In [ ]:
def filtrar_palavras_chave(x):
  words = []
  for i in x:
    if i in palavras_chave:
      words.append(i)
  return words

In [ ]:
# Aplicando a função filtrar palavras chaves
smd['keywords'] = smd['keywords'].apply(filtrar_palavras_chave)

In [ ]:
# Aplica a lisa de palavras o stemmer que deixa somente o radical
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
# Deixa as palavras minusculas e remove os espaços em palavras compostas 
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [ ]:
# Criando sopa de palavras unindo as colunas 
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']

In [ ]:
# Une as palavras da lista separando por espaços
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [ ]:
# Ele é usado para transformar um determinado texto em um vetor com base na frequência (contagem) 
# de cada palavra que ocorre em todo o texto.
count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
# fit_transform -> Aprenda o dicionário de vocabulário e retorne a matriz documento-termo.
#       Isso é equivalente a fit seguido de transformação, mas implementado de forma mais eficiente.
count_matrix = count.fit_transform(smd['soup'])

In [ ]:
# Calcule a similaridade de cosseno entre as amostras em X e Y.
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Cria uma coluna index contendo os indices do df - reset_index() -> Redefina o índice ou um nível dele.
smd = smd.reset_index()
# Cria uma serie com os titulos 
titles = smd['title']
# Faço o mapa reverso onde o id é o titulo e os valores são o id do filme
# Series -> Array unidimensional com rótulos de eixo (incluindo séries temporais).
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
indices

#**FILTRAGEM COLABORATIVA**

In [ ]:
reader = Reader()

In [ ]:
avaliacoes = pd.read_csv('ratings_small.csv')
# ds_avaliacoes vai ser usado no KNN
ds_avaliacoes = avaliacoes
avaliacoes.head()

In [ ]:
dados = Dataset.load_from_df(avaliacoes[['userId', 'movieId', 'rating']], reader)

In [ ]:
svd = SVD()
cross_validate(svd, dados, measures=['RMSE', 'MAE'], verbose=True)

In [ ]:
trainset = dados.build_full_trainset()
svd.fit(trainset)

In [ ]:
avaliacoes[avaliacoes['userId'] == 1]

In [ ]:
svd.predict(1, 302, 3)

# **FUNÇÕES UTILITARIAS**

In [ ]:
# Método que normaliza valores entre 0 e 1
# Recebe uma coluna como serie
def normalizacao_valores(serie):
  serie_index = serie.index
  array = serie.values
  y = array.reshape(-1, 1)
  scaler = MinMaxScaler(feature_range= (0, 1))
  rescaled = scaler.fit_transform(y)
  array = rescaled
  serie_normalizada = pd.DataFrame(array, index=serie_index)
  return serie_normalizada

In [ ]:
# Aqui as vezes retorna um int e as vezes retorna uma serie tratei dessa forma
def get_index(title):
  # Df indices contem o titulo e o código do filme ao lado
  idx = indices[title]

  if isinstance(idx, pd.Series):
    return idx[0]

  return idx

# **RECOMENDADOR KNN**  

Antes de qualquer coisa, ajustar o Dataset e organizar as colunas que irei utilizar o dataset de exemplo que irei me basear é esse:   
(https://github.com/krishnaik06/Recommendation_complete_tutorial/tree/master/KNN%20Movie%20Recommendation)  
Assim que estiver em Pedrinhas tenho que deixar o dataset que tenho conforme será utilizado aqui, realizar o pre-processado com os dados do meu Dataset.  
***Esse algoritmo utiliza a estratégia de filtragem colaborativa***  
Existem dois tipos:
* Baseado em itens;   
* Baseado em Usuários.  

***No nosso caso iremos utilizar a filtragem colaborativa baseado em usuários***

In [ ]:
def recomendacao_KNN(usuarioId, ds_filmes=ds_filmes, ds_links=ds_links, ds_avaliacoes=ds_avaliacoes):

  # Importando dataset
  # ds_filmes = pd.read_csv('movies_metadata.csv', dtype={'imdb_id':"string"})
  # ds_links = pd.read_csv('links_small.csv', dtype={'movieId':"Int64",'imdbId':"Int64",'tmdbId':"Int64"})
  # ds_avaliacoes = pd.read_csv('ratings_small.csv')

  # Cria serie com tmdbId não nulos de DS_Links
  # serie_tmdbid_ds_links = ds_links[ds_links['tmdbId'].notnull()]['tmdbId'].astype('int')

  # Remoção de linhas mal formatadas 
  # ds_filmes = ds_filmes.drop([19730, 29503, 35587])

  # Converte o tipo da coluna
  # ds_filmes['id'] = ds_filmes['id'].astype('int')

  # Criando serie com filmes filtrando por serie_tmdbid_ds_links
  # ds_filmes = ds_filmes[ds_filmes['id'].isin(serie_tmdbid_ds_links)]

  # Verificando imdbids nulos
  ds_filmes[ds_filmes['imdb_id'].isnull()]

  # Verificando ids nulos
  ds_filmes[ds_filmes['id'].isnull()]

  # Extraindo coluna imdbId para pegar o código do filme em links
  ds_filmes['imdbId'] = ds_filmes['imdb_id'].apply(lambda x: x if pd.isna(x) else str(x)[2:])

  # Alterando tipo da coluna
  ds_filmes['imdbId'] = ds_filmes['imdbId'].astype('int')

  # Merge pegando os movieIds de Links
  ds_filmes = pd.merge(ds_filmes, ds_links, on='imdbId')

  # Removendo a coluna tmdbId
  ds_filmes.drop(['tmdbId'], axis=1, inplace=True)
  
  # Para fazer a filtragem colaborativa o que me interessa é o Id do Filme e o titulo
  # Selecionando/filtrandos as colunas que irei utilizar 
  ds_filmes = ds_filmes[['movieId', 'title']]

  # Filtrando as colunas que preciso
  ds_avaliacoes = ds_avaliacoes[['userId', 'movieId', 'rating']]
  
  # Juntar as duas bases, semelhante ao join no SQL 
  df_filmes_avaliacoes = pd.merge(ds_filmes, ds_avaliacoes, on='movieId')
  

  # Pega o id do usuário e coloca como indice da coluna e pivotar a tabela 
  # Onde os titulos apareçam em cima e as avaliações serão os valores 
  df_recommender = df_filmes_avaliacoes.pivot_table(index='userId', columns='title', values='rating').fillna(0)
  

  # Modelo dos vizinhos próximos - KNN
  # Distância do cosseno não sofre muito com a maldição da dimensionalidade
  # Instancia o modelo KNN, usando a metrica do cosseno(hiperparametros)
  modelo_knn = NearestNeighbors(metric='cosine')
  # Calcula todas as distâncias em relação aos vizinhos próximos
  modelo_knn.fit(df_recommender)

  # Quantidade de vizinhos mais próximos que quero
  qtde_vizinhos = 4

  # Recupera a linha que o usuário se encontra
  # Usuario que eu quero recomendar algo
  idx_usuario_knn = df_recommender.index.get_loc(df_recommender.loc[usuarioId].name)
  # Quando chamar o modelo de KNN preciso saber a distância dos vizinhos próximos
  # E quais são os indices vizinhos próximos
  # Irei passar no parâmetro de kneighbors() o registro que quero usar para puxar
  # Vizinho mais próximo - Fiz uma mudança ao invés de iloc usei loc
  distancia_vizinhos, indices_vizinhos = modelo_knn.kneighbors(df_recommender.iloc[idx_usuario_knn].values.reshape(1, -1), n_neighbors=qtde_vizinhos)

  usuario = df_recommender.index[idx_usuario_knn]

  # Cria a lista que irá conter os dataframes
  dfs_vizinhos_proximo = []

  # For para percorrer cada um da lista 
  # Usei o flatter para remover uma dimensão 
  for i in range(0, len(distancia_vizinhos.flatten())):
    # Pula o primeiro elemento pois vai ser ele mesmo
    if i == 0:
      ds_usuario_knn = df_recommender.loc[usuario].to_frame()
      dfs_vizinhos_proximo.append(ds_usuario_knn)
    else:
      # Localiza o vizinho mais próximo em df_recommender
      vizinho_proximo = df_recommender.index[indices_vizinhos.flatten()[i]]
      # Adiciona o vizinho próximo a lista dos dataframes
      dfs_vizinhos_proximo.append(df_recommender.loc[vizinho_proximo].to_frame())
  

  # Pegando o indice dos vizinhos mais próximos
  idx_vizinhos_prox = [col for i in range(len(dfs_vizinhos_proximo)) for col in dfs_vizinhos_proximo[i] if i != 0]
  
  # Faz o merge da dataframe do usuário com seus vizinhos
  ds_titulos = reduce(lambda df_esq, df_dir: pd.merge(df_esq, df_dir, on=['title']), dfs_vizinhos_proximo)

  # Ordena de forma descrescente pelos vizinhos mais próximos
  ds_titulos = ds_titulos.sort_values(by=idx_vizinhos_prox, ascending=False)
  
  # Filtrar todos os titulos do vizinho que foram > 0 e onde usuário não assistiu
  ds_titulos = ds_titulos[((ds_titulos[idx_vizinhos_prox[0]] > 0) | (ds_titulos[idx_vizinhos_prox[1]] > 0) | (ds_titulos[idx_vizinhos_prox[2]] > 0)) & (ds_titulos[usuario] == 0)]
  
  # Criando e calculando a média 
  ds_titulos['Media'] = (ds_titulos[idx_vizinhos_prox[0]] + ds_titulos[idx_vizinhos_prox[1]] + ds_titulos[idx_vizinhos_prox[2]]) / len(idx_vizinhos_prox)

  # Resetando o indice
  ds_titulos.reset_index(inplace=True)

  # Atribui a média os valores de média normalizados 
  ds_titulos['est'] = normalizacao_valores(ds_titulos['Media'])

  # Filtrando as colunas que irei devolver ao método híbrido 
  titulos_recomendacao_KNN = ds_titulos[['title', 'est']]
  titulos_recomendacao_KNN = titulos_recomendacao_KNN.sort_values(by=['est'], ascending=False)


  return titulos_recomendacao_KNN.head(10)


**Testando Recomendador KNN**

In [ ]:
recomendacao_KNN(1)

#**RECOMENDADOR HÍBRIDO**

In [ ]:
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [ ]:
# Carregando o arquivo que contem os ids dos filmes, selecionando 
# as colunas 'movieId', 'tmdbId'
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
# Converte a coluna id_map['tmdbId'] para inteiro
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
# Mudando os nomes da coluna tmdbId para id
id_map.columns = ['movieId', 'id']
# Fazendo o merge de id_map com SMD
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [ ]:
indices_map = id_map.set_index('id')

In [ ]:
# Junta e soma as estimativas de cada dataframe 
def agrupa_recomendacoes(dataframe1, dataframe2):
  dfs_concatenados = pd.concat([dataframe1, dataframe2], ignore_index=True)
  dfs_agrupados = dfs_concatenados.groupby('title')[['est']].mean()
  # dfs_agrupados = dfs_concatenados.groupby('title')['est'].sum()

  dfs_agrupados = dfs_agrupados.sort_values('est', ascending=False)
  dfs_agrupados = dfs_agrupados.reset_index()

  return dfs_agrupados


In [ ]:
def recomendador_hibrido(userId, title):

  if isinstance(userId, int) and isinstance(title, str):

    if (userId in avaliacoes['userId'].values) and (title in id_map.index) :
      
      idx = get_index(title)

      titulos_recomendados_KNN = recomendacao_KNN(userId)
  
      sim_scores = list(enumerate(cosine_sim[int(idx)]))
      sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
      sim_scores = sim_scores[1:26]
      movie_indices = [i[0] for i in sim_scores]

      movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
      movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
      movies['est'] = normalizacao_valores(movies['est'])
      movies = movies[['title', 'est']]

      movies = movies.sort_values('est', ascending=False)

      movies = movies.head(10)

      recomendacao_hibrida = agrupa_recomendacoes(titulos_recomendados_KNN, movies)

      return recomendacao_hibrida
    
    else:
      print('Erro: O id do usuário ou nome do filme não consta no dataset')
  else:
    print('Algo inesperado aconteceu. Tente novamente!')

In [ ]:
recomendador_hibrido(1, "Avatar")

In [ ]:
recomendador_hibrido(500, 'Avatar')